## Sample Data Generation

This notebook sends the CARS4U Action training table to the model for scoring, creating enough data in the payload log for the monitors to display in the dashboard.

In [ ]:

from ibmdbpy import IdaDataBase, IdaDataFrame

# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# Update the two lines below with your own Db2 instance credentials
username = 'DASH***'
dsn = 'DASHDB;Database=BLUDB;Hostname=dashdb-entry-yp***;Port=50000;PROTOCOL=TCPIP;UID=dash***;PWD=***'
table = username + '.CAR_RENTAL_TRAINING'

idadb = IdaDataBase(dsn='DASHDB;' + dsn)

data_df = IdaDataFrame(idadb, table).as_dataframe()
data_df.head()

# You can close the database connection with the following code. Please keep the comment line with the @hidden_cell tag,
# because the close function displays parts of the credentials.
# @hidden_cell
# idadb.close()
# To learn more about the ibmdby package, please read the documentation: http://pythonhosted.org/ibmdbpy/


In [ ]:
import urllib3, requests, json

# retrieve your wml_service_credentials_username, wml_service_credentials_password, and wml_service_credentials_url from the
# Service credentials associated with your IBM Cloud Watson Machine Learning Service instance

wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "username": "***",
    "password": "***"
}

# retrieve the scoring endpoint url from the Implementation tab of your deployed
# CARS4U - Action Model Deployment
# It will look something like this:
# https://us-south.ml.cloud.ibm.com/v3/wml_instances/a6bqa815-mfrf-4501-a181-7292aad31162/deployments/fba78132-c468-48d5-a4f0-c226d40e453d/online
scoring_endpoint = '***'

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

fields = ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status','Car_Owner', 'Customer_Service', 'Satisfaction', 'Business_Area', 'Action']
values = json.loads(data_df.to_json(orient='values'))
payload_scoring = {"fields": fields, "values": values}

response_scoring = requests.post(scoring_endpoint, json=payload_scoring, headers=header)
print("Scoring complete")